In [1]:
import pandas as pd
import re
import numpy as np
org_df=pd.read_csv("suicide-blasts-dataset (1).csv")

df=org_df.copy(deep=True)

In [2]:
pd.set_option("display.max_columns",None)
df.set_index("S#",inplace=True)
df


,Date,Year,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,Location,Location Category,Location Sensitivity,Open/Closed Space,Influencing Event/Event,Target Type,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
S#,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Sunday-November 19-1995,1995,25 Jumaada al-THaany 1416 A.H,Holiday,Weekend,NaN,Islamabad,33.718000,73.0718,Capital,Egyptian Embassy,Foreign,High,Closed,NaN,Foreigner,NaN,14.0,15.0,NaN,60,2.0,NaN,NaN,15.835,60.503
2,Monday-November 6-2000,2000,10 SHa`baan 1421 A.H,Working Day,NaN,NaN,Karachi,24.991800,66.9911,Sindh,office of Nawa-e-Waqt,Office Building,Low,Closed,NaN,Media,NaN,NaN,3.0,NaN,3,1.0,NaN,NaN,23.770,74.786
3,Wednesday-May 8-2002,2002,25 safar 1423 A.H,Working Day,NaN,7:45 AM,Karachi,24.991800,66.9911,Sindh,Pakistan Navy bus Parked outside Five Star She...,Hotel,Medium,Closed,NaN,Foreigner,Christian,13.0,15.0,20.0,40,1.0,2.5 Kg,1.Jinnah Postgraduate Medical Center 2. Civil ...,31.460,88.628
4,Friday-June 14-2002,2002,3 Raby` al-THaany 1423 A.H,Working Day,NaN,11:10:00 AM,Karachi,24.991800,66.9911,Sindh,US Consulate Civil Lines Area,Foreign,High,Closed,NaN,Foreigner,Christian,NaN,12.0,NaN,51,1.0,NaN,NaN,31.430,88.574
5,Friday-July 4-2003,2003,4 Jumaada al-awal 1424 A.H,Working Day,NaN,NaN,Quetta,30.209500,67.0182,Balochistan,Imambargah MeCongy Road Quetta,Religious,Medium,Closed,during Friday prayer,Religious,Shiite,44.0,47.0,NaN,65,1.0,NaN,1.CMH Quetta\n2.Civil Hospital 3. Boland Medic...,33.120,91.616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,Thursday-October-05-2017,2017,13 MuHarram 1439 A.H,Working Day,NaN,NaN,Quetta,28.571051,67.496895,Balochistan,Fateh Pur Shrine in Jhal Magsi village,Religious,High,Open/Closed,at the entrance of shrine,Civilian,NaN,21.0,22.0,25.0,NaN,1.0,NaN,"Larkana Hospital, Gandawah District Headquarte...",27.800,81.000
493,Thursday-October-19-2017,2017,27 MuHarram 1439 A.H,Working Day,NaN,8:00 AM,Quetta,NaN,NaN,Balochistan,Sariab Mill area,Highway,Low,Open,explosives-laden vehicle rammed into a securit...,Police_Rangers,NaN,7.0,7.0,22.0,22,1.0,NaN,NaN,26.000,79.000
494,Thursday-November-9-2017,2017,19 Safar 1439 A.H,Working Day,NaN,7:00 AM,Quetta,30.221057,67.002524,Balochistan,Quetta's Chaman Housing Scheme on airport road,Police_Rangers,Low,Open,AIG Hamid Shakil’s vehicle,Police_Rangers,NaN,2.0,2.0,8.0,8,1.0,15KG,"Combined Military Hospital (CMH)-(Quetta), Civ...",25.000,77.000


In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 496 entries, 1 to 496
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Date                     496 non-null    object 
 1   Year                     496 non-null    int64  
 2   Islamic Date             342 non-null    object 
 3   Blast Day Type           486 non-null    object 
 4   Holiday Type             72 non-null     object 
 5   Time                     285 non-null    object 
 6   City                     496 non-null    object 
 7   Latitude                 493 non-null    float64
 8   Longitude                493 non-null    object 
 9   Province                 496 non-null    object 
 10  Location                 493 non-null    object 
 11  Location Category        460 non-null    object 
 12  Location Sensitivity     460 non-null    object 
 13  Open/Closed Space        461 non-null    object 
 14  Influencing Event/Event  191 no

In [4]:
df.shape

(496, 26)

In [5]:
df[df.duplicated()]

,Date,Year,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,Location,Location Category,Location Sensitivity,Open/Closed Space,Influencing Event/Event,Target Type,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
S#,,,,,,,,,,,,,,,,,,,,,,,,,,


# Short Summary of Data Set

The "Suicide Bomb Blast Dataset" contains detailed records of 496 suicide bombings in Pakistan spanning from 1995 to 2017. It includes 26 columns covering  contextual information about each incident. Key variables include date, year, Islamic date, city, province, location details, target type, number of blasts, casualties (killed and injured), explosive weight, hospital information, and weather conditions at the time of the blast. The dataset also highlights socio-religious dimensions such as holidays, targeted sects, and influencing events. With both quantitative (casualty counts, temperature, coordinates) and qualitative (target type, location sensitivity, open/closed space) features, this dataset provides a comprehensive foundation for analyzing patterns, trends, and impacts of suicide bombings in Pakistan.

# Columns Description

Date – The exact calendar date when the bomb blast occurred.

Year – The year of the incident.

Islamic Date – The corresponding date in the Islamic (Hijri) calendar.

Blast Day Type – The day of the week (e.g., Friday, Monday) or special type of day.

Holiday Type – Whether the blast happened on a public, religious, or national holiday.

Time – The time of day when the explosion occurred.

City – The city in Pakistan where the blast took place.

Latitude – Geographic latitude coordinate of the blast location.

Longitude – Geographic longitude coordinate of the blast location.

Province – Province of Pakistan where the incident occurred.

Location – Specific area or landmark of the blast (e.g., market, mosque).

Location Category – Type of location (e.g., urban, rural, commercial area).

Location Sensitivity – Security sensitivity of the location (e.g., high-profile, public).

Open/Closed Space – Whether the blast occurred in an open or enclosed environment.

Influencing Event/Event – Related event or trigger influencing the attack (e.g., political rally, protest).

Target Type – The main target (e.g., civilians, security forces, religious sites).

Targeted Sect if any – If applicable, the religious sect specifically targeted (e.g., Shia, Sunni).

Killed Min – Minimum number of people killed in the blast.

Killed Max – Maximum number of people killed in the blast.

Injured Min – Minimum number of people injured.

Injured Max – Maximum number of people injured.

No. of Suicide Blasts – Number of suicide bombers involved in the incident.

Explosive Weight (max) – Estimated maximum weight of explosives used.

Hospital Names – Hospitals where victims were taken.

Temperature (C) – Temperature in Celsius on the day of the blast.

Temperature (F) – Temperature in Fahrenheit on the day of the blast.

# 1) Issues with the dataset


1. `Dirty Data`

Table – Suicide_Bomb_Blasts

Date → not in datetime format, inconsistent sequence (validity, consistency)

Islamic Date → many null values (completion)

Blast Day Type → some missing values, not stored as category (completion, validity)

Time → many missing values, mix of text (“Evening”) and exact times (“1:30:00 PM”), inconsistent format (completion, consistency)

Latitude → few missing values (completion)

Longitude → some missing values, stored as object instead of numeric (completion, validity)

Province → case inconsistency e.g., FATA vs. fata (consistency)

Location → 3 rows missing, inconsistent text entries (completion, consistency)

Location Category → missing values, case inconsistency, duplicate categories under different names, not categorical type (completion, consistency, validity)

Location Sensitivity → missing values, case inconsistency, not categorical type (completion, consistency, validity)

Open/Closed Space → missing values, case inconsistency, not categorical type (completion, consistency, validity)

Influencing Event/Event → majority missing (completion)

Target Type → inconsistent case, duplicate categories under different names, not categorical type (consistency, validity)

Targeted Sect if any → mostly missing, case inconsistency, not categorical type (completion, consistency, validity)

Killed Min / Killed Max / Injured Min → some missing, stored as float though should be integer (completion, validity)

Injured Max → missing values, stored as object instead of numeric (completion, validity)

No. of Suicide Blasts → some missing, stored as float instead of integer (completion, validity)

Explosive Weight (max) → majority missing, string manipulation needed, inconsistent format (completion, consistency, validity)

Hospital Names → majority missing, duplicate hospital names, inconsistent spelling (completion, consistency)

2. `Messy Data`

Table – Suicide_Bomb_Blasts

Time column mixes categorical labels (Morning, Evening) with exact times (tidiness)

Explosive Weight (max) column contains text + numbers instead of a clean numeric field (tidiness)

Hospital Names column stores multiple hospitals in one cell and duplicates (tidiness)

Killed Min / Killed Max / Injured Min / Injured Max could be restructured into a single “Killed (Range)” and “Injured (Range)” column (tidiness)

# Date

In [6]:
# code
df["Date"]=df["Date"].str.findall(r"[a-zA-Z]*[,/\-\s]\d{1,2}[,/\-\s]\d{4}").str[0]
df["Date"]=pd.to_datetime(df["Date"],format="mixed")

In [7]:
# test
df["Date"]

S#
1     1995-11-19
2     2000-11-06
3     2002-05-08
4     2002-06-14
5     2003-07-04
         ...    
492   2017-10-05
493   2017-10-19
494   2017-11-09
495   2017-11-24
496   2017-11-25
Name: Date, Length: 496, dtype: datetime64[ns]

# Blast Day Type /Holiday Type

In [8]:
# code
df["Blast Day Type"]=df["Blast Day Type"].astype("category")
df["Holiday Type"]=df["Holiday Type"].str.lower().replace({"eid ul azha holiday":"eid-ul-azha","christmas/ birthday of quaid-e-azam":"christmas/birthday of quaid-e-azam"})
df["Holiday Type"]=df["Holiday Type"].astype("category")

In [9]:
# test
df["Holiday Type"].value_counts()

Holiday Type
weekend                               45
ashura                                 4
eid milad un-nabi                      3
eid-ul-fitar                           3
labour day                             3
ashura holiday                         2
christmas/birthday of quaid-e-azam     2
eid holidays                           2
eid-ul-azha                            2
iqbal day                              2
pakistan day                           2
defence day                            1
general elections                      1
Name: count, dtype: int64

# City column 


In [10]:
# code
df["City"]=df["City"].str.lower().str.strip()
df["City"]=df["City"].replace({
    "tangi, charsaddadistrict":"charsadda",
    "shabqadar-charsadda":"charsadda",
    "ghallanai, mohmand agency":"mohmand agency",
    "mosal kor, mohmand agency":"mohmand agency",
    "south waziristan":"waziristan",
    "north waziristan":"waziristan",
    "upper dir":"dir",
    "lower dir":"dir",
})
df['City']=df['City'].astype("category")

In [11]:
# test
df["City"].value_counts()

City
peshawar          75
quetta            38
waziristan        35
swat              30
karachi           29
lahore            25
bannu             24
islamabad         20
rawalpindi        19
d.g khan          17
hangu             17
khyber agency     15
bajaur agency     13
kohat             12
charsadda         11
mohmand agency    11
mardan             9
tank               8
dara adam khel     6
dir                6
kuram agency       6
lakki marwat       5
nowshehra          5
buner              4
attock             4
muzaffarabad       3
sargodha           3
malakand           3
shikarpur          3
swabi              3
poonch             2
shangla            2
orakzai agency     2
zhob               2
multan             2
chakwal            2
chaman             2
gujrat             2
lasbela            2
sukkur             1
bhakkar            1
wagah              1
totalai            1
tirah valley       1
taunsa             1
taftan             1
fateh jang         1
sudhanot

# Longitude

In [12]:
# code
df["Longitude"]=df["Longitude"].astype("float")

In [13]:
# test
df["Longitude"].dtype

dtype('float64')

# Provinces 

In [14]:
# code
df["Province"]=df["Province"].str.lower()
df["Province"]=df["Province"].astype("category")

In [15]:
# test
df["Province"].value_counts()

Province
kpk            251
fata            73
punjab          64
balochistan     47
sindh           35
capital         20
ajk              6
Name: count, dtype: int64


# Location Category

In [16]:
# code
df["Location Category"]=df["Location Category"].str.lower().replace({
    "mobile":"attack on Convoy",
    "commercial_residence":"Markets",  
})
df["Location Category"]=df["Location Category"].astype("category")

In [17]:
# test
df["Location Category"].value_counts()

Location Category
police_rangers      92
attack on Convoy    70
military            70
religious           57
market              40
parks_grounds       32
residence           27
government          21
hotel               10
office building      9
foreign              8
educational          6
transport            5
hospital             5
bank                 4
Markets              2
airport              1
highway              1
Name: count, dtype: int64

# Time 

In [18]:
# code
def clean_time(time):
    clean=None
    if not pd.isna(time):
        clean=",".join(re.findall(r"\d{1,2}[:.]\d{1,2}[:.]*\d*\s*[amp]*",time,flags=re.I))
    if clean:
        return clean
    return time
    

df["Time"]=df["Time"].apply(clean_time)


In [19]:
# test
df["Time"]

S#
1              NaN
2              NaN
3          7:45 AM
4      11:10:00 AM
5              NaN
          ...     
492            NaN
493        8:00 AM
494        7:00 AM
495        7:00 AM
496        7:00 AM
Name: Time, Length: 496, dtype: object

# Location Sensitivity



In [20]:
# code
df["Location Sensitivity"]=df["Location Sensitivity"].str.lower()
df["Location Sensitivity"]=df["Location Sensitivity"].astype("category")

In [21]:
# test
df["Location Sensitivity"].value_counts()

Location Sensitivity
high      264
low       121
medium     75
Name: count, dtype: int64

# Open/Closed Space

In [22]:
# code
df["Open/Closed Space"]=df["Open/Closed Space"].str.lower()
df["Open/Closed Space"]=df["Open/Closed Space"].astype("category")

In [23]:
# test
df["Open/Closed Space"].value_counts()

Open/Closed Space
open           333
closed         127
open/closed      1
Name: count, dtype: int64

# Target Type

In [24]:
df["Target Type"]=df["Target Type"].str.strip().str.lower()
df["Target Type"]=df["Target Type"].replace({
   "shia sect":"religious" ,
    "frontier corps":"military",
    "civilian judges":"judges_lawyers",
    "advocates_lawyers":"judges_lawyers",
    "children_women":"civilian",
    "army":"military"
})
df["Target Type"]=df["Target Type"].astype("category")

In [25]:
# test
df["Target Type"].value_counts()

Target Type
military               119
police_rangers         119
civilian                86
government official     52
religious               45
anti_militants          26
foreigner               17
judges_lawyers           3
media                    2
unknown                  1
Name: count, dtype: int64

# Targeted Sect if any

In [26]:
# code
df["Targeted Sect if any"]=df["Targeted Sect if any"].str.lower()
df["Targeted Sect if any"]=df["Targeted Sect if any"].astype("category")

In [27]:
# test
df["Targeted Sect if any"].value_counts()

Targeted Sect if any
shiite          47
sunni           38
christian        9
ahmedi           1
jews             1
shiite_sunni     1
Name: count, dtype: int64

In [28]:
# code
df["Killed Min"]=df["Killed Min"].astype("Int64")
df["Killed Max"]=df["Killed Max"].astype("Int64")
df["Injured Min"]=df["Injured Min"].astype("Int64")
df["Injured Max"]=pd.to_numeric(df["Injured Max"],errors="coerce")
df["Injured Max"]=df["Injured Max"].astype("Int64")
df["No. of Suicide Blasts"]=df["No. of Suicide Blasts"].astype("Int64")


In [29]:
# test
df[["Killed Min","Killed Max","Injured Min","Injured Max","No. of Suicide Blasts"]]

,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts
S#,,,,,
1,14,15,<NA>,60,2
2,<NA>,3,<NA>,3,1
3,13,15,20,40,1
4,<NA>,12,<NA>,51,1
5,44,47,<NA>,65,1
...,...,...,...,...,...
492,21,22,25,<NA>,1
493,7,7,22,22,1
494,2,2,8,8,1


In [30]:
# code
def int_conversion(values):
    if not pd.isna(values):
        if "-" in values:
            d1,d2=values.split("-")
            return (float(d1)+float(d2))/2
        else:
            return float(values)
    return 0
df["Explosive Weight (max)"]=df["Explosive Weight (max)"].str.replace(r"[a-zA-Z]","",regex=True).str.strip().str.replace(r"\s+","-",regex=True).apply(int_conversion)
df.rename(columns={"Explosive Weight (max)":"Explosive Weight (max)(kg)"},inplace=True)




In [31]:
# test
df["Explosive Weight (max)(kg)"].unique()

array([   0. ,    2.5,   30. ,    2. ,   12.5,    5. ,   10. ,    5.5,
         15. ,    3. ,    6.5,    4.5,   50. ,  100. ,  150. , 1000. ,
          8. ,   60. ,   40. ,    7.5,   85. ,  125. ,  160. ,  500. ,
         24. ,   20. ,   12. ,  180. ,   70. ,   16. ,    7. ,  400. ,
        200. ,   55. ,  250. ,  600. ,   14. ,  300. ,    6. , 1200. ,
          9. ,   18. ,  450. ,  105. ,  700. ,   22.5,   27.5,   45. ,
         13. ,   75. ,   25. ])

In [32]:
# code

hos_name=df["Hospital Names"].str.strip().str.lower().str.replace(r"[\d\n.]","",regex=True).str.split(r"/|-|and",regex=True)
df["Hospital Names"]=df["Hospital Names"].str.strip().str.lower().str.replace(r"[\d\n.]","",regex=True).str.split(r"/|-|and",regex=True)



In [33]:
# test
df["Hospital Names"]

S#
1                                                    NaN
2                                                    NaN
3      [jinnah postgraduate medical center  civil hos...
4                                                    NaN
5      [cmh quettacivil hospital  bol,  medical complex]
                             ...                        
492    [larkana hospital, g, awah district headquarte...
493                                                  NaN
494    [combined military hospital (cmh), (quetta), c...
495                [hayatabad medical complex(peshawar)]
496                             [civil hospital(quetta)]
Name: Hospital Names, Length: 496, dtype: object

In [34]:
df.to_csv("cleaned_explosions.csv", index=False)